In [4]:
hidden_sizes = [16]
num_layers = 1

for i, hidden_size in enumerate(hidden_sizes):
    print(i)
    print(hidden_size)

0
16


In [ ]:
import random
import numpy

def decimal_to_gray_decimal(n):
    return n ^ (n >> 1)

def gray_to_binary(n):
    n = int(n, 2) # convert to int
    mask = n
    while mask != 0:
        mask >>= 1
        n ^= mask # n=n^mask
    return format(n, pixel_depth_format)


pixel_depth = 8
pixel_depth_format = '0{}b'.format(pixel_depth)
lower_bound = 0
upper_bound = pow(2,pixel_depth)-1

print('max input: {}'.format(upper_bound))
input_as_decimal = random.randint(lower_bound,upper_bound)
print('random input: {}'.format(input_as_decimal))
input_as_gray_decimal = decimal_to_gray_decimal(input_as_decimal)
print('decimal input to gray code decimal: {}'.format(input_as_gray_decimal))
input_as_gray_code = format(input_as_gray_decimal, pixel_depth_format)
print('decimal input to gray code: {}'.format(input_as_gray_code))
input_as_binary = gray_to_binary(input_as_gray_code)
print('input converted to binary:', input_as_binary)
print('type: {}'.format(type(input_as_binary)))
x = numpy.frombuffer()(input_as_binary, dtype=numpy.int)
print('type: {}'.format(type(x)))
print(x)

In [34]:
import numpy
a = '1100'
b = numpy.array(a)
print(b)

1100


In [41]:
import numpy as np
pixel_depth = 4
pixel_depth_format = '0{}b'.format(pixel_depth)

def decimal_to_gray_decimal(n):
    return n ^ (n >> 1)

def gray_to_binary(n):
    n = int(n, 2) # convert to int
    mask = n
    while mask != 0:
        mask >>= 1
        n ^= mask # n=n^mask
    return format(n, pixel_depth_format)

def getSample(pixel_depth, test_flag):
    lower_bound = 0
    upper_bound = pow(2, pixel_depth) - 1

    input_as_decimal = random.randint(lower_bound, upper_bound)
    input_as_gray_decimal = decimal_to_gray_decimal(input_as_decimal)
    input_as_gray_code = format(input_as_gray_decimal, pixel_depth_format)
    input_as_binary = gray_to_binary(input_as_gray_code)
    input_as_gray_code = np.array(list(input_as_gray_code[::-1]), dtype=int)
    input_as_binary = np.array(list(input_as_binary[::-1]), dtype=int)

    if test_flag == 1:
        print('Sample as gray: {}, sample as binary: {}'.fomart(input_as_gray_code, input_as_binary))

    return input_as_gray_code, input_as_binary

x, y = getSample(pixel_depth, test_flag=0)
print(x,y)
print(type(x))

[1 0 0 0] [1 0 0 0]
<class 'numpy.ndarray'>


In [4]:
import numpy as np

x = np.array([[1,0,1,0],[0,1,0,1],
              [1,0,1,1],[0,1,1,0],[1,1,0,0]])
print(x)

[[1 0 1 0]
 [0 1 0 1]
 [1 0 1 1]
 [0 1 1 0]
 [1 1 0 0]]
